In [ ]:
teams_train = pd.read_csv(f"{DATA_PATH}teams_train.csv")
teams_test = pd.read_csv(f"{DATA_PATH}teams_test.csv")
players_train = pd.read_csv(f"{DATA_PATH}players_train.csv")
players_test = pd.read_csv(f"{DATA_PATH}players_test.csv")

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [ ]:
players_train.head(3)

,gameid,league,split,playoffs,date,game,patch,side,position,playername,teamname,champion,ban1,ban2,ban3,ban4,ban5,gamelength,result,kills,deaths,assists,doublekills,triplekills,quadrakills,pentakills,firstblood,firstbloodkill,firstbloodassist,firstbloodvictim,team kpm,ckpm,barons,opp_barons,inhibitors,opp_inhibitors,damagetochampions,dpm,damageshare,damagetakenperminute,damagemitigatedperminute,wardsplaced,wpm,wardskilled,wcpm,controlwardsbought,visionscore,vspm,totalgold,earnedgold,earned gpm,earnedgoldshare,goldspent,total cs,minionkills,monsterkills,cspm,goldat10,xpat10,csat10,opp_goldat10,opp_xpat10,opp_csat10,golddiffat10,xpdiffat10,csdiffat10,killsat10,assistsat10,deathsat10,opp_killsat10,opp_assistsat10,opp_deathsat10,goldat15,xpat15,csat15,opp_goldat15,opp_xpat15,opp_csat15,golddiffat15,xpdiffat15,csdiffat15,killsat15,assistsat15,deathsat15,opp_killsat15,opp_assistsat15,opp_deathsat15
0,ESPORTSTMNT01_2700815,LCK,Spring,0,2022-01-12 6:20,1,12.01,Blue,top,Kingen,DRX,Graves,Diana,Caitlyn,Twisted Fate,LeBlanc,Viktor,2195,0,0,5,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1367,0.5194,0.0,0.0,0.0,1.0,18483,505.2301,0.369379,474.5877,645.1298,11,0.3007,5,0.1367,9,35,0.9567,13189,8435,230.5695,0.210996,12875,315.0,270.0,45,8.6105,3170.0,4619.0,83.0,3223.0,4640.0,89.0,-53.0,-21.0,-6.0,0.0,0.0,0.0,0.0,0.0,0.0,4763.0,6943.0,127.0,4965.0,7658.0,135.0,-202.0,-715.0,-8.0,0.0,0.0,1.0,0.0,1.0,0.0
1,ESPORTSTMNT01_2700815,LCK,Spring,0,2022-01-12 6:20,1,12.01,Blue,jng,Pyosik,DRX,Viego,Diana,Caitlyn,Twisted Fate,LeBlanc,Viktor,2195,0,1,3,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1367,0.5194,0.0,2.0,0.0,1.0,3799,103.8451,0.075922,640.0456,568.0182,13,0.3554,15,0.4100,13,58,1.5854,10792,6038,165.0478,0.151037,10375,178.0,27.0,151,4.8656,3075.0,3131.0,63.0,3036.0,3180.0,64.0,39.0,-49.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,5107.0,5355.0,89.0,4867.0,5626.0,104.0,240.0,-271.0,-15.0,1.0,2.0,0.0,1.0,0.0,1.0
2,ESPORTSTMNT01_2700815,LCK,Spring,0,2022-01-12 6:20,1,12.01,Blue,mid,Zeka,DRX,Ryze,Diana,Caitlyn,Twisted Fate,LeBlanc,Viktor,2195,0,0,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1367,0.5194,0.0,0.0,0.0,0.0,7658,209.3303,0.153044,490.6879,271.6811,19,0.5194,5,0.1367,7,39,1.0661,14226,9472,258.9157,0.236936,14075,351.0,304.0,47,9.5945,3361.0,4914.0,92.0,3326.0,4770.0,91.0,35.0,144.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,5589.0,7592.0,144.0,5151.0,7796.0,151.0,438.0,-204.0,-7.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
from datetime import datetime
import pytz

In [ ]:
league_locations = {
    "LCK": "Asia/Seoul",
    "LEC": "Europe/Berlin",
    "LCS": "America/Los_Angeles",
    "CBLOL": "America/Sao_Paulo",
    "PCS": "Asia/Taipei",
    "VCS": "Asia/Ho_Chi_Minh",
    "MSI": {
        2022: "Asia/Seoul",
        2023: "Europe/London",
        2024: "Asia/Shanghai"
    },
    "WLDs": {
        2022: "America/Los_Angeles",
        2023: "Asia/Seoul",
        2024: "Europe/Berlin"
    }
}


In [ ]:
for i in range(len(players_train)):
    date_str = players_train.loc[i, 'date']

    # 시간 형식이 한 자리수인지 확인하고 앞에 0 추가
    if len(date_str.split(' ')[1].split(':')[0]) == 1:
        date_str = date_str.replace(' ', ' 0', 1)

    # 초가 없는 경우 ':00' 추가
    if len(date_str.split(':')) == 2:  # 초가 없는 경우
        date_str += ':00'

    # 수정된 날짜를 다시 저장
    players_train.loc[i, 'date'] = date_str


In [ ]:
# 시간 변환 준비 (UTC를 기준으로 시작)
utc = pytz.timezone('UTC')

# 결과 저장할 리스트
local_times = []

# 각 행에 대해 처리
for i in range(len(players_train)):
    utc_time_str = players_train.loc[i, 'date']  # UTC 시간 문자열
    league = players_train.loc[i, 'league']  # 리그 이름

    # UTC 시간을 datetime 객체로 변환
    utc_time = datetime.strptime(utc_time_str, '%Y-%m-%d %H:%M:%S')
    utc_time = utc.localize(utc_time)

    # 연도 추출
    year = utc_time.year

    # 리그의 시간대 가져오기
    if league in league_locations:
        if isinstance(league_locations[league], dict):  # MSI나 WLDs처럼 연도별로 다른 경우
            local_tz = pytz.timezone(league_locations[league].get(year, "UTC"))  # 연도 없으면 기본 UTC
        else:
            local_tz = pytz.timezone(league_locations[league])  # 일반 리그의 시간대
        local_time = utc_time.astimezone(local_tz)  # 로컬 시간대로 변환
        local_time_str = local_time.strftime('%Y-%m-%d %H:%M:%S')  # 문자열로 변환
    else:
        local_time_str = utc_time_str  # 시간대가 없으면 기본 UTC 사용

    # 결과 저장
    local_times.append(local_time_str)

# 변환된 시간 추가
players_train['local_time'] = local_times


In [ ]:
players_train.head(4)

,gameid,league,split,playoffs,date,game,patch,side,position,playername,teamname,champion,ban1,ban2,ban3,ban4,ban5,gamelength,result,kills,deaths,assists,doublekills,triplekills,quadrakills,pentakills,firstblood,firstbloodkill,firstbloodassist,firstbloodvictim,team kpm,ckpm,barons,opp_barons,inhibitors,opp_inhibitors,damagetochampions,dpm,damageshare,damagetakenperminute,damagemitigatedperminute,wardsplaced,wpm,wardskilled,wcpm,controlwardsbought,visionscore,vspm,totalgold,earnedgold,earned gpm,earnedgoldshare,goldspent,total cs,minionkills,monsterkills,cspm,goldat10,xpat10,csat10,opp_goldat10,opp_xpat10,opp_csat10,golddiffat10,xpdiffat10,csdiffat10,killsat10,assistsat10,deathsat10,opp_killsat10,opp_assistsat10,opp_deathsat10,goldat15,xpat15,csat15,opp_goldat15,opp_xpat15,opp_csat15,golddiffat15,xpdiffat15,csdiffat15,killsat15,assistsat15,deathsat15,opp_killsat15,opp_assistsat15,opp_deathsat15,local_time
0,ESPORTSTMNT01_2700815,LCK,Spring,0,2022-01-12 06:20:00,1,12.01,Blue,top,Kingen,DRX,Graves,Diana,Caitlyn,Twisted Fate,LeBlanc,Viktor,2195,0,0,5,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1367,0.5194,0.0,0.0,0.0,1.0,18483,505.2301,0.369379,474.5877,645.1298,11,0.3007,5,0.1367,9,35,0.9567,13189,8435,230.5695,0.210996,12875,315.0,270.0,45,8.6105,3170.0,4619.0,83.0,3223.0,4640.0,89.0,-53.0,-21.0,-6.0,0.0,0.0,0.0,0.0,0.0,0.0,4763.0,6943.0,127.0,4965.0,7658.0,135.0,-202.0,-715.0,-8.0,0.0,0.0,1.0,0.0,1.0,0.0,2022-01-12 15:20:00
1,ESPORTSTMNT01_2700815,LCK,Spring,0,2022-01-12 06:20:00,1,12.01,Blue,jng,Pyosik,DRX,Viego,Diana,Caitlyn,Twisted Fate,LeBlanc,Viktor,2195,0,1,3,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1367,0.5194,0.0,2.0,0.0,1.0,3799,103.8451,0.075922,640.0456,568.0182,13,0.3554,15,0.4100,13,58,1.5854,10792,6038,165.0478,0.151037,10375,178.0,27.0,151,4.8656,3075.0,3131.0,63.0,3036.0,3180.0,64.0,39.0,-49.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,5107.0,5355.0,89.0,4867.0,5626.0,104.0,240.0,-271.0,-15.0,1.0,2.0,0.0,1.0,0.0,1.0,2022-01-12 15:20:00
2,ESPORTSTMNT01_2700815,LCK,Spring,0,2022-01-12 06:20:00,1,12.01,Blue,mid,Zeka,DRX,Ryze,Diana,Caitlyn,Twisted Fate,LeBlanc,Viktor,2195,0,0,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1367,0.5194,0.0,0.0,0.0,0.0,7658,209.3303,0.153044,490.6879,271.6811,19,0.5194,5,0.1367,7,39,1.0661,14226,9472,258.9157,0.236936,14075,351.0,304.0,47,9.5945,3361.0,4914.0,92.0,3326.0,4770.0,91.0,35.0,144.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,5589.0,7592.0,144.0,5151.0,7796.0,151.0,438.0,-204.0,-7.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-01-12 15:20:00
3,ESPORTSTMNT01_2700815,LCK,Spring,0,2022-01-12 06:20:00,1,12.01,Blue,bot,Deft,DRX,Aphelios,Diana,Caitlyn,Twisted Fate,LeBlanc,Viktor,2195,0,3,2,2,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.1367,0.5194,0.0,0.0,0.0,0.0,15003,410.1048,0.299832,302.7062,462.9431,20,0.5467,8,0.2187,5,58,1.5854,16249,11495,314.2141,0.287539,14825,352.0,314.0,38,9.6219,3259.0,3186.0,86.0,3092.0,3109.0,80.0,167.0,77.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,7171.0,5691.0,140.0,4322.0,4431.0,116.0,2849.0,1260.0,24.0,2.0,2.0,0.0,0.0,0.0,2.0,2022-01-12 15:20:00


In [ ]:
list(players_train['local_time'].unique())

['2022-01-12 15:20:00',
 '2022-01-12 18:02:00',
 '2022-01-12 19:07:00',
 '2022-01-12 20:49:00',
 '2022-01-13 15:51:00',
 '2022-01-13 17:53:00',
 '2022-01-13 18:50:00',
 '2022-01-13 19:51:00',
 '2022-01-13 21:51:00',
 '2022-01-14 16:15:00',
 '2022-01-14 17:42:00',
 '2022-01-14 18:32:00',
 '2022-01-14 20:51:00',
 '2022-01-14 22:00:00',
 '2022-01-14 17:19:00',
 '2022-01-14 18:28:00',
 '2022-01-14 19:45:00',
 '2022-01-14 20:14:00',
 '2022-01-14 21:10:00',
 '2022-01-14 13:26:00',
 '2022-01-14 15:05:00',
 '2022-01-14 16:00:00',
 '2022-01-14 17:14:00',
 '2022-01-14 18:13:00',
 '2022-01-15 16:01:00',
 '2022-01-15 17:40:00',
 '2022-01-15 18:32:00',
 '2022-01-15 20:43:00',
 '2022-01-15 16:15:00',
 '2022-01-15 17:12:00',
 '2022-01-15 18:13:00',
 '2022-01-15 19:08:00',
 '2022-01-15 20:34:00',
 '2022-01-15 12:52:00',
 '2022-01-15 14:02:00',
 '2022-01-15 14:50:00',
 '2022-01-15 16:14:00',
 '2022-01-15 17:14:00',
 '2022-01-16 15:26:00',
 '2022-01-16 17:43:00',
 '2022-01-16 18:46:00',
 '2022-01-16 19: